# Dependencias 

In [ ]:
%pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

DNE_URL = os.getenv('DNE_URL')
CORREIOS_SYNC_REPOSITORY = os.getenv('CORREIOS_SYNC_REPOSITORY')
MINDB_IMAGE = os.getenv('MINDB_IMAGE')
JAVA_IMAGE = os.getenv('JAVA_IMAGE')

print(f"""
  DNE_URL: {DNE_URL}
  Correios Sync Repository: {CORREIOS_SYNC_REPOSITORY}
  MinDB Image: {MINDB_IMAGE}
  Java Builder Image: {JAVA_IMAGE}"""
)

# Baixando o DNE do repositório dos Correios

In [ ]:
%%bash -s "$DNE_URL"

#!/bin/bash

rm -rf eDNE_Basico
curl -o eDNE_Basico.zip $1
unzip eDNE_Basico.zip -d eDNE_Basico
unzip eDNE_Basico/eDNE_Basico*.zip -d eDNE_Basico
rm eDNE_Basico.zip eDNE_Basico/eDNE*.zip

rm -rf converted_files/
mkdir converted_files

EDNE_BASICO_PATH=./eDNE_Basico/Delimitado
iconv -f ISO-8859-1 -t UTF-8 ${EDNE_BASICO_PATH}/LOG_BAIRRO.TXT > ./converted_files/LOG_BAIRRO.TXT
iconv -f ISO-8859-1 -t UTF-8 ${EDNE_BASICO_PATH}/LOG_GRANDE_USUARIO.TXT > ./converted_files/LOG_GRANDE_USUARIO.TXT
iconv -f ISO-8859-1 -t UTF-8 ${EDNE_BASICO_PATH}/LOG_LOCALIDADE.TXT > ./converted_files/LOG_LOCALIDADE.TXT
iconv -f ISO-8859-1 -t UTF-8 ${EDNE_BASICO_PATH}/LOG_UNID_OPER.TXT > ./converted_files/LOG_UNID_OPER.TXT
iconv -f ISO-8859-1 -t UTF-8 ${EDNE_BASICO_PATH}/LOG_CPC.TXT > ./converted_files/LOG_CPC.TXT

for i in ${EDNE_BASICO_PATH}/LOG_LOGRADOURO_* ; do
  filename=$(echo "$i" | rev | cut -d / -f 1 | rev)
  iconv -f ISO-8859-1 -t UTF-8 "$i" > ./converted_files/"$filename"
done

rm -rf eDNE_Basico
echo "Done."

# Executando o correios sync

In [ ]:
%%bash -s "$CORREIOS_SYNC_REPOSITORY"
git clone $1 correios-sync
mv converted_files correios-sync

In [ ]:
%%bash -s "$MINDB_IMAGE"
cd correios-sync

cp src/main/resources/nightfall.properties.sample src/main/resources/nightfall.properties

mkdir -p sql
cat <<EOF > sql/ref_cep_info.sql
ALTER TABLE ref_cep_info ADD COLUMN created_at datetime NOT NULL DEFAULT CURRENT_TIMESTAMP;
ALTER TABLE ref_cep_info ADD COLUMN updated_at datetime NULL;
EOF

cat <<EOF > compose.yml
services:
  mindb:
    container_name: mindb
    image: $1
    ports:
      - "3306:3306"
    volumes:
      - ./sql/ref_cep_info.sql:/root/dml/marketplace/init.sql
      - elo7_mindb:/var/lib/mysql

volumes:
  elo7_mindb:

networks:
  default:
    name: development
EOF

docker compose up -d

Na sequencia iremos executar a importação no container com spark atraves do comando:

In [ ]:
%%bash -s "$JAVA_IMAGE"
cd correios-sync

docker run --rm -i \
  --network=development \
  -v ${PWD}:/home/correios-sync \
  -w /home/correios-sync \
  --name correios-sync \
  $1 bash -c "mvn clean compile && mvn scala:run -DmainClass=com.elo7.postalCode.CorreiosApplication"

Execução concluida, vamos ver a versão

In [47]:
%%bash
docker compose -f correios-sync/compose.yml exec mindb \
  mysql -u root elo7 \
  -e "SELECT COUNT(cep) FROM ref_cep_info WHERE version = 2;"

COUNT(cep)
0
